# Blcok Recognition

## Import Libraries

In [2]:
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import open3d as o3d
import copy
import pickle

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
with open('/home/hr/Desktop/dep.p', 'rb') as dep:
    d = pickle.load(dep, encoding="16UC1")
    
d

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [4]:
with open('/home/hr/Desktop/rgb.p', 'rb') as rgb:
    r = pickle.load(rgb)
    
r

array([[[ 20,  40,  21],
        [ 38,  58,  39],
        [ 50,  68,  51],
        ...,
        [109, 111,  81],
        [113, 115,  85],
        [108, 110,  80]],

       [[ 53,  67,  49],
        [ 56,  70,  52],
        [ 58,  72,  54],
        ...,
        [ 85,  92,  65],
        [ 85,  93,  63],
        [ 81,  89,  59]],

       [[ 62,  72,  55],
        [ 58,  68,  51],
        [ 56,  66,  49],
        ...,
        [ 78,  88,  65],
        [ 76,  86,  63],
        [ 75,  85,  62]],

       ...,

       [[ 42,  49,  28],
        [ 42,  49,  28],
        [ 43,  50,  29],
        ...,
        [156,  76,  47],
        [159,  76,  44],
        [158,  75,  43]],

       [[ 43,  50,  29],
        [ 42,  49,  28],
        [ 42,  49,  28],
        ...,
        [154,  73,  46],
        [156,  71,  45],
        [156,  71,  45]],

       [[ 42,  49,  28],
        [ 42,  49,  28],
        [ 42,  49,  28],
        ...,
        [149,  69,  46],
        [152,  66,  46],
        [152,  66,  46]]

In [5]:
d.min()

0

In [6]:
d1 = np.asarray(d, dtype = np.float32)

In [7]:
d2 = d1/915
d2 = np.array(d2, dtype=np.float32)
d2.shape

(1536, 2048)

In [8]:
d3 = np.array(d2 * 255, dtype='uint8')

In [9]:
# cv2.imshow('d2', d3)
# cv2.imwrite('../imgs/depth.png', d3)
# # cv2.imshow('r', r)
# # cv2.imwrite('../imgs/color.png', r)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

## Read Test Images

In [10]:
# img_color = cv2.imread('test_imgs/jenga_tower_color.png')
# img_depth = cv2.imread('test_imgs/jenga_tower_depth.png', 0)
# img_color = cv2.imread('/home/shs/Desktop/rgb1.png')
# img_depth = cv2.imread('/home/shs/Desktop/dep.png', 0)
img_color = r
img_depth = d
print('color shape: ', img_color.shape)
print('depth shape: ', img_depth.shape)

cv2.imshow(f'color', img_color)
# cv2.imwrite('../imgs/red_extract.png', each_color_filtered)
cv2.waitKey(0)
cv2.destroyAllWindows()


color shape:  (1536, 2048, 3)
depth shape:  (1536, 2048)


## Extract Blocks' Masks By Colors

In [11]:
height, width = img_color.shape[:2] # 이미지의 높이와 너비 불러옴, 가로 [0], 세로[1]

img_hsv = cv2.cvtColor(img_color, cv2.COLOR_BGR2HSV) # cvtColor 함수를 이용하여 hsv 색공간으로 변환

In [12]:
colors = ['green', 'pink', 'yellow', 'blue', 'violet', 'red']

In [13]:
# RED
lower_red1 = np.array([0, 100, 50])
upper_red1 = np.array([15, 255, 255])
lower_red2 = np.array([160,100,50])
upper_red2 = np.array([179,255,255])

# PINK
lower_pink1 = np.array([0, 15, 200])
upper_pink1 = np.array([5, 90, 255])
lower_pink2 = np.array([150,15,200])
upper_pink2 = np.array([179,80,255])

# GREEN
lower_green = (70-20, 100, 100)
upper_green = (70+15, 255, 255)

# YELLOW
lower_yellow = (30-10, 60, 120)
upper_yellow = (30+10, 255, 255)

# BLUE
lower_blue = (100-10, 100, 100)
upper_blue = (100+9, 255, 255)

# VIOLET
lower_violet = (130-20, 60, 60)
upper_violet = (130+30, 120, 130)

In [14]:
blocks_rgb_by_color = []
blocks_mask_by_color = []
for color in colors:
    if color == 'pink' or color =='red':
        for i in (1,2):
            exec(f"lower_color{i} = lower_{color}{i}")
            exec(f"upper_color{i} = upper_{color}{i}")

        mask_color1 = cv2.inRange(img_hsv, lower_color1, upper_color1)
        mask_color2 = cv2.inRange(img_hsv, lower_color2, upper_color2)
        img_mask_color = mask_color1 + mask_color2
        
        # Denoise by Erosion and Dilation
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=2)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=2)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 
        
        exec(f"img_result_{color} = img_result_color")
    
    else:
        exec(f"lower_color = lower_{color}")
        exec(f"upper_color = upper_{color}")

        img_mask_color = cv2.inRange(img_hsv, lower_color, upper_color) # 범위내의 픽셀들은 흰색, 나머지 검은색

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=1)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=1)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 

        exec(f"img_result_{color} = img_result_color")
        
    _, src_bin = cv2.threshold(img_mask_color, 0, 255, cv2.THRESH_OTSU)
    each_color_filtered = cv2.bitwise_and(img_color, img_color, mask = src_bin)
    
    if color == 'violet':
        cv2.imshow('src_bin', src_bin)
        # cv2.imwrite('../imgs/red_mask.png', src_bin)
        cv2.imshow(f'{color}_filtered', each_color_filtered)
        # cv2.imwrite('../imgs/red_extract.png', each_color_filtered)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

    blocks_color = []
    blocks_mask = []

    for i in range(1, cnt): # 각각의 객체 정보에 들어가기 위해 반복문. 범위를 1부터 시작한 이유는 배경을 제외
        (x, y, w, h, area) = stats[i]
        cen_x, cen_y = map(int, centroids[i])
        block_mask = (labels==i)*img_mask_color
        block_color = cv2.bitwise_and(img_color, img_color, mask = block_mask)
        
        # 노이즈 제거
        if area < 300:
            continue
        
        if color == 'violet':
            cv2.imshow('blk clr', block_color)
            # cv2.imwrite('../imgs/red_block2_color.png', block_color)
            cv2.imshow('blk msk', block_mask)
            # cv2.imwrite('../imgs/red_block2_mask.png', block_mask)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
        
        blocks_color.append(block_color)
        blocks_mask.append(block_mask)
        
        
    exec(f"blocks_rgb_{color} = blocks_color")
    exec(f"blocks_mask_{color} = blocks_mask")
    exec(f"blocks_rgb_by_color.append(blocks_rgb_{color})")
    exec(f"blocks_mask_by_color.append(blocks_mask_{color})")
    

In [15]:
tower_mask = 0
tower_color = 0
for mask, color in zip(blocks_mask_by_color, blocks_rgb_by_color):
    for block_m in mask:
        tower_mask += block_m
    
    for block_c in color:
        tower_color += block_c

In [16]:
cv2.imshow('tower mask', tower_mask)
# cv2.imwrite('../imgs/tower_mask.png', tower_mask)
cv2.imshow('tower color', tower_color)
# cv2.imwrite('../imgs/tower_color.png', tower_color)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
for c, b, rgb_m in zip(colors, blocks_mask_by_color, blocks_rgb_by_color):
    print(c)
    print(len(b), "Blocks")

green
6 Blocks
pink
6 Blocks
yellow
6 Blocks
blue
6 Blocks
violet
6 Blocks
red
6 Blocks


## Get PointCloud from RGB Image + Depth Image

In [18]:
# temp intrinsic matrix
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.intrinsic_matrix = [[968.813, 0, 1023.83],
                              [0, 968.635, 775.975],
                              [0, 0, 1]]


# extrinsic = np.array([[0.999987, 0.00512943, 0.000573447, -32.0559],
#                      [-0.00516119, 0.994725, 0.102443, -1.91788],
#                      [-4.49478e-05, -0.102445, 0.994739, 3.99916],
#                      [0, 0, 0, 1]])
# extrinsic

In [19]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=50, origin=[0, 0, 0])
mesh_frame_small = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1, origin=[0, 0, 0])

In [20]:
def get_pointcloud_from_color_depth(color_image, depth_image, intrinsic):
    o3d_img = o3d.geometry.Image()
    
    if isinstance(color_image, type(o3d_img)):
        pass
    elif isinstance(color_image, np.ndarray):
        color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        color_image = o3d.geometry.Image(color_image)
        
    if isinstance(depth_image, type(o3d_img)):
        pass
    elif isinstance(depth_image, np.ndarray):
        depth_image = o3d.geometry.Image(depth_image)
        
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_image, depth_image, depth_scale=1, depth_trunc=3000.0, convert_rgb_to_intensity=False)
    # print(np.asarray(rgbd_image.depth).max())
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsic)
    
    return pcd

In [21]:
masked_depth = cv2.bitwise_and(img_depth, img_depth, mask = tower_mask)

In [22]:
tower_pcd = get_pointcloud_from_color_depth(color_image=tower_color, depth_image=masked_depth, intrinsic=intrinsic)

In [23]:
print(tower_pcd)
print(np.asarray(tower_pcd.points))
o3d.visualization.draw_geometries([tower_pcd])

PointCloud with 67399 points.
[[-56.16928138  11.67134679 491.        ]
 [-55.5491101   11.64757623 490.        ]
 [-55.04333654  11.64757623 490.        ]
 ...
 [ -8.35539986 190.77294337 454.        ]
 [ -7.90415694 191.19314809 455.        ]
 [ -7.43451007 191.19314809 455.        ]]


In [24]:
points = np.asarray(tower_pcd.points)

print("X min/max:", np.min(points[:, 0]), np.max(points[:, 0]))
print("Y min/max:", np.min(points[:, 1]), np.max(points[:, 1]))
print("Z min/max:", np.min(points[:, 2]), np.max(points[:, 2]))

X min/max: -134.2438633668211 26.797493427524177
Y min/max: 11.14839439004371 334.89281308232717
Z min/max: 453.0 842.0


In [30]:
blocks_pcd_by_color = []
all_pcd = []
for color, block_mask in zip(colors, blocks_mask_by_color):
    # print(color)
    # if color != 'blue':
    #     continue
    blocks_pcd = []
    for msk in block_mask:
        masked_block_rgb = cv2.bitwise_and(tower_color, tower_color, mask = msk)
        masked_block_depth = cv2.bitwise_and(img_depth, img_depth, mask = msk)
        
        # Get Each Block's PointCloud
        pcd = get_pointcloud_from_color_depth(color_image=masked_block_rgb, depth_image=masked_block_depth, intrinsic=intrinsic)
        
        # Remove Outlier Points
        pcd, _ = pcd.remove_radius_outlier(256, 25)
        blocks_pcd.append(pcd)
        all_pcd.append(pcd)
        
        # if color=='blue':
        #     o3d.visualization.draw_geometries([pcd])
    
    exec(f"blocks_pcd_{color} = blocks_pcd")
    exec(f"blocks_pcd_by_color.append(blocks_pcd_{color})")

In [31]:
len(blocks_pcd_by_color)

6

## Tower ICP

In [32]:
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(all_pcd)):
    pcd_combined += all_pcd[point_id]

print(pcd_combined)
# print(len(pcd_combined.points))

PointCloud with 66900 points.


In [33]:
points = np.asarray(pcd_combined.points)

print("X min/max:", np.min(points[:, 0]), np.max(points[:, 0]))
print("Y min/max:", np.min(points[:, 1]), np.max(points[:, 1]))
print("Z min/max:", np.min(points[:, 2]), np.max(points[:, 2]))

X min/max: -77.80314673729607 26.797493427524177
Y min/max: 11.14839439004371 194.67784562812616
Z min/max: 453.0 533.0


In [ ]:

# o3d.visualization.draw_geometries([pcd_combined, mesh_frame_small])

o3d.visualization.draw_geometries([pcd_combined, mesh_frame])

In [122]:
mesh_tower = o3d.io.read_triangle_mesh("mesh/jenga_tower_side_xy.stl")
print(mesh_tower)

mesh_tower.compute_vertex_normals()
# o3d.visualization.draw_geometries([mesh])

pcd_target = mesh_tower.sample_points_uniformly(number_of_points=len(pcd_combined.points))
print(pcd_target)
o3d.visualization.draw_geometries([pcd_target])

TriangleMesh with 12 points and 4 triangles.
PointCloud with 62835 points.


In [123]:
np.linalg.inv(np.asarray([[0, 0, -1, 0],
            [1, 0, 0, 0],
            [0, -1, 0, 0],
            [0, 0, 0, 1]]))

array([[ 0.,  1.,  0.,  0.],
       [-0., -0., -1., -0.],
       [-1., -0., -0., -0.],
       [ 0.,  0.,  0.,  1.]])

In [124]:
initial_transform = np.asarray([[0, 0, -1, 0],
                                [-1, 0, 0, 0],
                                [0, 1, 0, 0],
                                [0, 0, 0, 1]])

In [125]:
def prepare_icp(source, target):
    source_tmp = copy.deepcopy(source)
    target_tmp = copy.deepcopy(target)
    
    # make the point cloud into right position
    source_tmp.transform(initial_transform)
    
    # resize the target pointcloud to make two pointclouds into same scale
    # resize =(np.array(target_tmp.points)[:,2].max() - np.array(target_tmp.points)[:,2].min()) / (np.array(source_tmp.points)[:,2].max() - np.array(source_tmp.points)[:,2].min())
    
    # move the source pcd to do icp
    move = np.array(target_tmp.get_oriented_bounding_box().get_center() - source_tmp.get_oriented_bounding_box().get_center())
    
    # a = o3d.cpu.pybind.utility.Vector3dVector(np.array(target_tmp.points))
    # b = o3d.cpu.pybind.utility.Vector3dVector(np.array(source_tmp.points))

    source_tmp.transform(np.linalg.inv(initial_transform))
    
    # target_tmp.points = a
    # source_tmp.points = b
    
    o3d.visualization.draw_geometries([source_tmp, target_tmp])
    
    return source_tmp, target_tmp, move

In [126]:
source, target,  move = prepare_icp(pcd_combined, pcd_target)

In [ ]:
move

array([445.99156197, -56.46618907,  11.93535865])

In [ ]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    # source_temp.paint_uniform_color([1, 0.706, 0])
    # target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp, mesh_frame])

In [ ]:
threshold = 10
trans_init = np.asarray([[0, 0, -1, move[0]],
                         [-1, 0, 0, move[1]],
                         [0, 1, 0, move[2]],
                         [0, 0, 0, 1]])
draw_registration_result(source, target, trans_init)

KeyboardInterrupt: 

In [ ]:
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, trans_init)
print(evaluation)

Initial alignment
RegistrationResult with fitness=4.634700e-01, inlier_rmse=5.178779e+00, and correspondence_set size of 71709
Access transformation to get result.


In [ ]:
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)

RegistrationResult with fitness=1.000000e+00, inlier_rmse=1.036098e+00, and correspondence_set size of 154722
Access transformation to get result.
Transformation is:
[[ 6.90824136e-01 -6.77474195e-02 -7.19841858e-01  2.69430804e+02]
 [-7.22900480e-01 -4.64043844e-02 -6.89392144e-01  2.26432627e+02]
 [ 1.33007205e-02  9.96622757e-01 -8.10319108e-02  5.15018238e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
draw_registration_result(source, target, reg_p2p.transformation)

In [ ]:
source_temp = copy.deepcopy(source)

source_temp.transform(reg_p2p.transformation)


PointCloud with 154722 points.

In [ ]:
(np.array(source_temp.points))[:,2].min()

-2.446097732659119

## Blocks Transformation

In [ ]:
trans1 = reg_p2p.transformation

In [ ]:
trans = copy.deepcopy(reg_p2p.transformation)

In [ ]:
print(trans)
trans[0,3]/=1000
trans[1,3]/=1000
trans[2,3]/=1000

print(np.linalg.inv(trans))

[[ 6.90824136e-01 -6.77474195e-02 -7.19841858e-01  2.69430804e+02]
 [-7.22900480e-01 -4.64043844e-02 -6.89392144e-01  2.26432627e+02]
 [ 1.33007205e-02  9.96622757e-01 -8.10319108e-02  5.15018238e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 0.69082414 -0.72290048  0.01330072 -0.02312606]
 [-0.06774742 -0.04640438  0.99662276 -0.02256718]
 [-0.71984186 -0.68939214 -0.08103191  0.35422174]
 [ 0.          0.          0.          1.        ]]


In [ ]:
def transform_blocks(pcd, icp_transform):
    pcd_temp = copy.deepcopy(pcd)
    aa = o3d.cpu.pybind.utility.Vector3dVector(np.array(pcd_temp.points))
    pcd_temp.points = aa
    pcd_temp.transform(icp_transform)
    
    return pcd_temp

In [ ]:
# WHAT IS THE TARGET BLOCK?
target_block_color = 'red'
target_block_label = 3

In [ ]:
for col, pcds in zip(colors, blocks_pcd_by_color):
    if col != target_block_color:
        continue
    print(col)
    
    for idx, pcd in enumerate(pcds):
        if idx != target_block_label:
            continue
        
        # print("--------------------------------")
        print(idx)

        new_trans = copy.deepcopy(trans)
        new_trans[0,3]*=1000
        new_trans[1,3]*=1000
        new_trans[2,3]*=1000

        pcd_new = transform_blocks(pcd, new_trans)
        
        # o3d.visualization.draw_geometries([pcd_new, mesh_frame, pcd_new.get_axis_aligned_bounding_box()])
        
        box_extent = pcd_new.get_axis_aligned_bounding_box().get_extent()
        print("BOX EXTENT : ", box_extent)
        
        center_coordinate = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points()).mean(axis=0)
        
        # print(np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points()))
        print("BOX CENTER COORDINATE : ", center_coordinate)
        
        # print("BOX MAX X,Y and MEAN Z Coordinate")
        x_mean = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,0].mean()
        y_mean = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,1].mean()
        z_mean = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,2].mean()
        
        if box_extent[1] > 70:
            print("PULL DIRECTION : X")
            vector = np.array([-1, 0, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean #- 25/2
            cen_y = y_mean #- 75/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.00001, cen_y - 0.00001, cen_z - 0.00001], [cen_x + 0.00001, cen_y + 0.00001, cen_z + 0.00001]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x + 150
            target_y = cen_y
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.00001, target_y - 0.00001, target_z - 0.00001], [target_x + 0.00001, target_y + 0.00001, target_z + 0.00001]]))
            print("----------------------------------")
            
        elif box_extent[0] > 70:
            print("PULL DIRECTION : Y")
            vector = np.array([0, -1, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean# - 75/2
            cen_y = y_mean# - 25/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.00001, cen_y - 0.00001, cen_z - 0.00001], [cen_x + 0.00001, cen_y + 0.00001, cen_z + 0.00001]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x
            target_y = cen_y + 150
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.00001, target_y - 0.00001, target_z - 0.00001], [target_x + 0.00001, target_y + 0.00001, target_z + 0.00001]]))
            print("----------------------------------")
            
        elif abs(center_coordinate[0]) < 10 and box_extent [1] < 15:
            print("PUSH DIRECTION : Y or -Y")
            vector = np.array([0, -1, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean# - 25/2
            cen_y = y_mean - 75/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.00001, cen_y - 0.00001, cen_z - 0.00001], [cen_x + 0.00001, cen_y + 0.00001, cen_z + 0.00001]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x
            target_y = cen_y + 150
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.00001, target_y - 0.00001, target_z - 0.00001], [target_x + 0.00001, target_y + 0.00001, target_z + 0.00001]]))
            print("----------------------------------")
            
        elif abs(center_coordinate[1]) < 10 and box_extent [0] < 15:
            print("PUSH DIRECTION : X or -X")
            vector = np.array([-1, 0, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean - 75/2
            cen_y = y_mean# - 25/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.00001, cen_y - 0.00001, cen_z - 0.00001], [cen_x + 0.00001, cen_y + 0.00001, cen_z + 0.00001]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x + 150
            target_y = cen_y
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.00001, target_y - 0.00001, target_z - 0.00001], [target_x + 0.00001, target_y + 0.00001, target_z + 0.00001]]))
            print("----------------------------------")
            
        elif box_extent[1] < 15:
            print("PULL DIRECTION : -X")
            vector = np.array([1, 0, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean# - 25/2
            cen_y = y_mean - 75/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.00001, cen_y - 0.00001, cen_z - 0.00001], [cen_x + 0.00001, cen_y + 0.00001, cen_z + 0.00001]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x - 150
            target_y = cen_y
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.00001, target_y - 0.00001, target_z - 0.00001], [target_x + 0.00001, target_y + 0.00001, target_z + 0.00001]]))
            print("----------------------------------")
            
        elif box_extent[0] < 15:
            print("PULL DIRECTION : -Y")
            vector = np.array([0, 1, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean - 75/2
            cen_y = y_mean# - 25/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.00001, cen_y - 0.00001, cen_z - 0.00001], [cen_x + 0.00001, cen_y + 0.00001, cen_z + 0.00001]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x
            target_y = cen_y - 150
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.00001, target_y - 0.00001, target_z - 0.00001], [target_x + 0.00001, target_y + 0.00001, target_z + 0.00001]]))
            print("----------------------------------")
            
        else:
            print("NOTHING")
            
        mesh_COORD = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=center_coordinate)
        ptc1 = o3d.geometry.PointCloud()
        ptc1.points = pt1
        ptc1.paint_uniform_color([1, 0, 0])
        ptc2 = o3d.geometry.PointCloud()
        ptc2.points = pt2
        ptc2.paint_uniform_color([0, 0, 1])
        line = o3d.geometry.LineSet.create_from_point_cloud_correspondences(ptc1, ptc2, [(0,0)])
        o3d.visualization.draw_geometries([pcd_new, source_temp, mesh_frame, ptc1, ptc2, line])

red
3
BOX EXTENT :  [ 5.39557331 24.56136968 15.3656097 ]
BOX CENTER COORDINATE :  [ 38.87938771 -25.81394808  97.95376067]
PULL DIRECTION : -Y
-----BLOCK CENTER COORDINATE-----
X : 1.3793877101446412
Y : -25.813948083892853
Z : 97.95376066579404
-----TARGET COORDINATE-----
X : 1.3793877101446412
Y : -175.81394808389285
Z : 97.95376066579404
----------------------------------


In [ ]:
# mesh_block = o3d.io.read_triangle_mesh("mesh/single_block_x.stl")
# print(mesh_block)

# mesh_block.compute_vertex_normals()

In [ ]:
# # WHAT IS THE TARGET BLOCK?
# target_block_color = 'green'
# target_block_label = 2

In [ ]:
# for col, pcds in zip(colors, blocks_pcd_by_color):
#     if col != target_block_color:
#         continue
#     print(col)
    
#     for idx, pcd in enumerate(pcds):
#         if idx != target_block_label:
#             continue
        
#         # print("--------------------------------")
#         print(idx)
#         pcd_new = transform_blocks(pcd, trans)

#         z_mean = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,2].mean()

#         print(z_mean)
        
#         # o3d.visualization.draw_geometries([pcd_new, mesh_frame, pcd_new.get_axis_aligned_bounding_box()])
        
#         box_extent = pcd_new.get_axis_aligned_bounding_box().get_extent()
#         print("BOX EXTENT : ", box_extent)
        
#         center_coordinate = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points()).mean(axis=0)
        
#         # print(np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points()))
#         print("BOX CENTER COORDINATE : ", center_coordinate)
        
        
#         block_source = mesh_block.sample_points_uniformly(number_of_points=len(pcd_new.points))
#         print(block_source)
#         o3d.visualization.draw_geometries([block_source])
#         block_source, target_pcd, move = prepare_icp(source=block_source, target=pcd_new)
        
#         threshold = 10
#         trans_init = np.asarray([[1, 0, 0, move[0]],
#                                 [0, 1, 0, move[1]],
#                                 [0, 0, 1, move[2]],
#                                 [0, 0, 0, 1]])
        

#         draw_registration_result(block_source, target_pcd, trans_init)
    
#         print("Initial alignment")
#         evaluation = o3d.pipelines.registration.evaluate_registration(
#             block_source, target_pcd, threshold, trans_init)
#         print(evaluation)
        
#         reg_p2p = o3d.pipelines.registration.registration_icp(
#             block_source, target_pcd, threshold, trans_init,
#             o3d.pipelines.registration.TransformationEstimationPointToPoint(),
#             o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
#         print(reg_p2p)
#         print("Transformation is:")
#         print(reg_p2p.transformation)
    
#         draw_registration_result(block_source, target_pcd, reg_p2p.transformation)
        
#         print("NEW Transformation is:")
#         r11 = reg_p2p.transformation[0,0]
#         r21 = reg_p2p.transformation[1,0]

#         theta_z = np.arctan2(r21, r11)
#         trans_z = copy.deepcopy(reg_p2p.transformation)
#         trans_z[0,0] = np.cos(theta_z)
#         trans_z[0,1] = -np.sin(theta_z)
#         trans_z[0,2] = 0
#         trans_z[1,0] = np.sin(theta_z)
#         trans_z[1,1] = np.cos(theta_z)
#         trans_z[1,2] = 0
#         trans_z[2,0] = 0
#         trans_z[2,1] = 0
#         trans_z[2,2] = 1
        
        
#         print(trans_z)
    
#         draw_registration_result(block_source, target_pcd, trans_z)

#         pcd_new1 = transform_blocks(block_source, trans_z)
#         # pcd_new1 = 

        
        
#         # print("BOX MAX X,Y and MEAN Z Coordinate")
#         x_mean = np.array(pcd_new1.get_axis_aligned_bounding_box().get_box_points())[:,0].mean()
#         y_mean = np.array(pcd_new1.get_axis_aligned_bounding_box().get_box_points())[:,1].mean()
#         z_mean = np.array(pcd_new1.get_axis_aligned_bounding_box().get_box_points())[:,2].mean()
#         print(x_mean, y_mean, z_mean)
        
#         # if box_extent[1] > 70:
#         #     print("PULL DIRECTION : X")
#         #     vector = np.array([-1, 0, 0])
#         #     print("-----BLOCK CENTER COORDINATE-----")
#         #     cen_x = x_mean #- 25/2
#         #     cen_y = y_mean #- 75/2
#         #     cen_z = z_mean
#         #     print("X :", cen_x)
#         #     print("Y :", cen_y)
#         #     print("Z :", cen_z)
#         #     pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
#         #     print("-----TARGET COORDINATE-----")
#         #     target_x = cen_x + 100
#         #     target_y = cen_y
#         #     target_z = cen_z
#         #     print("X :", target_x)
#         #     print("Y :", target_y)
#         #     print("Z :", target_z)
#         #     pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
#         #     print("----------------------------------")
            
#         # elif box_extent[0] > 70:
#         #     print("PULL DIRECTION : Y")
#         #     vector = np.array([0, -1, 0])
#         #     print("-----BLOCK CENTER COORDINATE-----")
#         #     cen_x = x_mean# - 75/2
#         #     cen_y = y_mean# - 25/2
#         #     cen_z = z_mean
#         #     print("X :", cen_x)
#         #     print("Y :", cen_y)
#         #     print("Z :", cen_z)
#         #     pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
#         #     print("-----TARGET COORDINATE-----")
#         #     target_x = cen_x
#         #     target_y = cen_y + 100
#         #     target_z = cen_z
#         #     print("X :", target_x)
#         #     print("Y :", target_y)
#         #     print("Z :", target_z)
#         #     pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
#         #     print("----------------------------------")
            
#         # elif abs(center_coordinate[0]) < 10 and box_extent [1] < 20:
#         #     print("PUSH DIRECTION : Y or -Y")
#         #     vector = np.array([0, -1, 0])
#         #     print("-----BLOCK CENTER COORDINATE-----")
#         #     cen_x = x_mean# - 25/2
#         #     cen_y = y_mean - 75/2
#         #     cen_z = z_mean
#         #     print("X :", cen_x)
#         #     print("Y :", cen_y)
#         #     print("Z :", cen_z)
#         #     pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
#         #     print("-----TARGET COORDINATE-----")
#         #     target_x = cen_x
#         #     target_y = cen_y + 150
#         #     target_z = cen_z
#         #     print("X :", target_x)
#         #     print("Y :", target_y)
#         #     print("Z :", target_z)
#         #     pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
#         #     print("----------------------------------")
            
#         # elif abs(center_coordinate[1]) < 10 and box_extent [0] < 20:
#         #     print("PUSH DIRECTION : X or -X")
#         #     vector = np.array([-1, 0, 0])
#         #     print("-----BLOCK CENTER COORDINATE-----")
#         #     cen_x = x_mean - 75/2
#         #     cen_y = y_mean# - 25/2
#         #     cen_z = z_mean
#         #     print("X :", cen_x)
#         #     print("Y :", cen_y)
#         #     print("Z :", cen_z)
#         #     pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
#         #     print("-----TARGET COORDINATE-----")
#         #     target_x = cen_x + 150
#         #     target_y = cen_y
#         #     target_z = cen_z
#         #     print("X :", target_x)
#         #     print("Y :", target_y)
#         #     print("Z :", target_z)
#         #     pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
#         #     print("----------------------------------")
            
#         # elif box_extent[1] < 20:
#         #     print("PULL DIRECTION : -X")
#         #     vector = np.array([1, 0, 0])
#         #     print("-----BLOCK CENTER COORDINATE-----")
#         #     cen_x = x_mean# - 25/2
#         #     cen_y = y_mean - 75/2
#         #     cen_z = z_mean
#         #     print("X :", cen_x)
#         #     print("Y :", cen_y)
#         #     print("Z :", cen_z)
#         #     pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
#         #     print("-----TARGET COORDINATE-----")
#         #     target_x = cen_x - 100
#         #     target_y = cen_y
#         #     target_z = cen_z
#         #     print("X :", target_x)
#         #     print("Y :", target_y)
#         #     print("Z :", target_z)
#         #     pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
#         #     print("----------------------------------")
            
#         # elif box_extent[0] < 20:
#         #     print("PULL DIRECTION : -Y")
#         #     vector = np.array([0, 1, 0])
#         #     print("-----BLOCK CENTER COORDINATE-----")
#         #     cen_x = x_mean - 75/2
#         #     cen_y = y_mean# - 25/2
#         #     cen_z = z_mean
#         #     print("X :", cen_x)
#         #     print("Y :", cen_y)
#         #     print("Z :", cen_z)
#         #     pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
#         #     print("-----TARGET COORDINATE-----")
#         #     target_x = cen_x
#         #     target_y = cen_y - 100
#         #     target_z = cen_z
#         #     print("X :", target_x)
#         #     print("Y :", target_y)
#         #     print("Z :", target_z)
#         #     pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
#         #     print("----------------------------------")
            
#         # else:
#         #     print("NOTHING")
            
#         # mesh_COORD = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=center_coordinate)
#         # ptc1 = o3d.geometry.PointCloud()
#         # ptc1.points = pt1
#         # ptc1.paint_uniform_color([1, 0, 0])
#         # ptc2 = o3d.geometry.PointCloud()
#         # ptc2.points = pt2a
#         # ptc2.paint_uniform_color([0, 0, 1])
#         # line = o3d.geometry.LineSet.create_from_point_cloud_correspondences(ptc1, ptc2, [(0,0)])
#         # o3d.visualization.draw_geometries([pcd_new, source_temp, mesh_frame, ptc1, ptc2, line])

In [ ]:
o3d.visualization.draw_geometries([source_temp, mesh_frame, source_temp.get_axis_aligned_bounding_box()])

In [ ]:
np.array(source_temp.get_axis_aligned_bounding_box().get_box_points())

array([[-38.936143  , -40.32110286,  -2.44609773],
       [ 42.05933228, -40.32110286,  -2.44609773],
       [-38.936143  ,  41.71935903,  -2.44609773],
       [-38.936143  , -40.32110286, 181.9211539 ],
       [ 42.05933228,  41.71935903, 181.9211539 ],
       [-38.936143  ,  41.71935903, 181.9211539 ],
       [ 42.05933228, -40.32110286, 181.9211539 ],
       [ 42.05933228,  41.71935903,  -2.44609773]])

In [ ]:
np.array(source_temp.points)[:,2].max()

181.92115390385513

In [ ]:
np.array(pt1)-np.array(pt2)

array([[  0., 150.,   0.],
       [  0., 150.,   0.],
       [  0., 150.,   0.]])

In [ ]:
o3d.visualization.draw_geometries([pcd_new, mesh_frame])

In [ ]:
c = np.array([target_x, target_y, target_z, 1])

In [ ]:
c/1000

array([ 0.00137939, -0.17581395,  0.09795376,  0.001     ])

In [ ]:
vector

array([0, 1, 0])

In [ ]:
v = np.array([cen_x, cen_y, cen_z, 1])

In [ ]:
np.linalg.inv(trans)

array([[ 0.69082414, -0.72290048,  0.01330072, -0.02312606],
       [-0.06774742, -0.04640438,  0.99662276, -0.02256718],
       [-0.71984186, -0.68939214, -0.08103191,  0.35422174],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [ ]:
np.inner(np.linalg.inv(trans), c)

array([129.32863137, 105.66546793, 112.628655  ,   1.        ])

In [ ]:
c2 = np.inner(np.linalg.inv(trans), c)[:3]
v2 = np.inner(np.linalg.inv(trans), v)[:3]

In [ ]:
c2

array([129.32863137, 105.66546793, 112.628655  ])

In [ ]:
v2

array([20.89355932, 98.70481027,  9.21983335])

In [ ]:
tar_x, tar_y, tar_z = c2

In [ ]:
c2-v2

array([108.43507204,   6.96065766, 103.40882165])

In [ ]:
ptpt = o3d.cpu.pybind.utility.Vector3dVector(np.array([[tar_x, tar_y, tar_z], [tar_x - 0.00001, tar_y - 0.00001, tar_z - 0.00001], [tar_x + 0.00001, tar_y + 0.00001, tar_z + 0.00001]]))


In [ ]:
ptpt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[v2[0], v2[1], v2[2]], [v2[0] - 0.00001, v2[1] - 0.00001, v2[2] - 0.00001], [v2[0] + 0.00001, v2[1] + 0.00001, v2[2] + 0.00001]]))


In [ ]:
ptpcd = o3d.geometry.PointCloud()
ptpcd.points = ptpt

ptpcd2 = o3d.geometry.PointCloud()
ptpcd2.points = ptpt2

In [ ]:
o3d.visualization.draw_geometries([pcd_combined, ptpcd, ptpcd2, mesh_frame, pcd_target])

In [ ]:
mesh_tower = o3d.io.read_triangle_mesh("mesh/jenga_tower_side_xy.stl")
print(mesh_tower)

TriangleMesh with 12 points and 4 triangles.
